## Database: CITY VIEW

#----------------------------------------#

## DATABASE connection:

In [22]:
import numpy as np
import pandas as pd
import xarray as x
from configparser import ConfigParser
import sqlalchemy as sa # conection to the database

from sqlalchemy import create_engine, text
from datetime import datetime, timedelta



print (" import data")
### SET conection to MS-sql server:
################################################## SET postgre-sql connection:

################################################## read database keys:
def config(filename, section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception(
            'Section {0} not found in the {1} file'.format(section, filename))

    return db

keys = config(filename='database.ini')

POSTGRESQL_SERVER_NAME=keys['host']
PORT=                  keys['port']
Database_name =        keys['database']
USER =                 keys['user']
PSW =                  keys['password']
##################################################

engine_postgresql = sa.create_engine('postgresql://'+USER+':'+PSW+ '@'+POSTGRESQL_SERVER_NAME+':'+str(PORT)+ '/' + Database_name)
#print (engine_postgresql)
print ("Engine started..")

 import data
Engine started..


#### (1.1)  VIEW I

In [25]:
#### (1.1)  VIEW !


################ OUTPUT VIEW name:
schema = "cube"
name_of_view = "test_view_lux"



### calc_indicator EC1020I and Export to a new table inside the database:
query = ("""      

  CREATE OR REPLACE VIEW """+ schema +"."+ name_of_view + """
 AS

SELECT 
                0 as env_zone_code,
                cu_city_eurostat.index, 
                cu_city_eurostat.urau_code, 
                cu_city_eurostat.estat_code, 
                cu_city_eurostat.city_code, 
                cu_city_eurostat.city_code_version, 
                cu_city_eurostat.parameter, 
                cu_city_eurostat.lineage, 
                cu_city_eurostat.datasource, 
                cu_city_eurostat.year, 
                cu_city_eurostat.parameter_value, 

                city_working_lat_lon_list.lat,
                city_working_lat_lon_list.lon,
                city_working_lat_lon_list.iso2,
                l_city_urau2021.urau_name,
                l_city_urau2021.nuts3_2021 AS area_sqm

                FROM cube.cu_city_eurostat
                LEFT JOIN lut.city_working_lat_lon_list  ON cu_city_eurostat.city_code = city_working_lat_lon_list.city_code
                LEFT JOIN lut.l_city_urau2021            ON cu_city_eurostat.city_code = city_working_lat_lon_list.urau_code
                        
                WHERE cube.cu_city_eurostat.city_code ~~ 'LU0%'::text


                union


                SELECT
                        cu_city_env_zones.env_zone_code,
                        99 as index,
                        cu_city_env_zones.urau_code, 
                        '' as estat_code, 
                        cu_city_env_zones.city_code, 
                        cu_city_env_zones.city_code_version, 
                        cu_city_env_zones.parameter, 
                        cu_city_env_zones.lineage, 
                        cu_city_env_zones.datasource, 
                        'nix' as year,
                        cu_city_env_zones.parameter_value, 

                        city_working_lat_lon_list.lat,
                        city_working_lat_lon_list.lon,
                        city_working_lat_lon_list.iso2,
                        l_city_urau2021.urau_name,
                        l_city_urau2021.nuts3_2021 AS area_sqm


                        FROM cube.cu_city_env_zones
                        LEFT JOIN lut.city_working_lat_lon_list  ON cu_city_env_zones.city_code = city_working_lat_lon_list.city_code
                        LEFT JOIN lut.l_city_urau2021            ON cu_city_env_zones.city_code = city_working_lat_lon_list.urau_code
                        
                WHERE cube.cu_city_env_zones.city_code ~~ 'LU0%'::text;

  




         """)
#print (query)
connection = engine_postgresql.raw_connection()
cursor = connection.cursor()
cursor.execute(query)
connection.commit()

print ("VIEW 1 produced!! ")


VIEW 1 produced!! 
